In [4]:
import matlab.engine
import numpy as np
import pandas as pd
import os

from scipy import optimize
import sklearn.gaussian_process as skg
from scipy.special import erf, erfc
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from tqdm import tqdm
from numba import jit

import bokeh.io 
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.transform import linear_cmap
from bokeh.models import LinearColorMapper
bokeh.io.output_notebook()

Loading BokehJS ...

---
# Setting up MATLAB link for simulations

In [5]:
eng = matlab.engine.start_matlab()
eng.cd('.', nargout=0)
eng.cd('../matlab', nargout=0)

In [45]:
obs_loc = matlab.double([[2],[0.1]])
goal = matlab.double([[4],[0]])
alpha = 0.2

out_SD = eng.single_vs_double_integrator(obs_loc, goal, alpha)
out_U = eng.unicycle(obs_loc,goal,alpha)
out = {**out_SD, **out_U}
for x in out.keys():
    out[x] = np.asarray(out[x])

In [46]:
p1 = figure(match_aspect=True)
# p1.line(
#     out["xU"][:,0],
#     out["xU"][:,1],
#     line_width=3,
#     color="blue",
#     legend_label="Unicycle"
# )
p1.line(
    out["xD"][:,0],
    out["xD"][:,1],
    line_width=3,
    color="blue",
    legend_label="Double Integrator Dynamics"
)
p1.line(
    out["xS"][:,0],
    out["xS"][:,1],
    line_width=3,
    color="green",
    legend_label="Single Integrator Dynamics"
)

p1.circle(
    obs_loc[0],
    obs_loc[1],
    radius=0.02,
    color="black"
)

p1.circle(
    obs_loc[0],
    obs_loc[1],
    radius=0.5,
    fill_alpha=0,
    line_width=3,
    color="black"
)

p2 = figure(
    x_axis_label="Time",
    y_axis_label="CBF"
)
# p2.line(
#     out["tU"][:,0],
#     out["hU"][:,0],
#     line_width=3,
#     color="blue"
# )
p2.line(
    out["tD"][:,0],
    out["hD"][:,0],
    line_width=3,
    color="blue"
)
p2.line(
    out["tS"][:,0],
    out["hS"][:,0],
    line_width=3,
    color="green"
)

p2.line(
    [0,60],
    [0,0],
    line_width=3,
    color="black"
)

p1.legend.location = "bottom_left"

grid = gridplot([[p1,p2]])
show(grid)

In [8]:
np.min(out["hU"])

0.0010735990250620442

---
# Initialize GPA and define visualization function

In [9]:
lam = 1e-6
l = 0.5
kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
gpa = skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

In [10]:
def query_di(x):

    # print(x)
    obs_loc = matlab.double([[x[0]],[x[1]]])
    goal = matlab.double([[4],[0]])
    out_SD = eng.single_vs_double_integrator(obs_loc, goal, float(x[2]))
    
    return np.min(out_SD["hD"])

def query_uni(x):

    # print(x)
    obs_loc = matlab.double([[x[0]],[x[1]]])
    goal = matlab.double([[4],[0.5]])
    out = eng.unicycle(obs_loc, goal, float(x[2]))
    
    return np.min(out["hU"])

In [11]:
query_di([0.6,0.,1.])

-0.030500657955024102

In [12]:
%%capture
n0 = 20
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))
# a0 = np.random.uniform(0.1,1,(n0,1))
a0 = np.random.uniform(0.5,0.5,(n0,1))

X = np.hstack((x0,y0,a0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
gpa.fit(X,y)

In [13]:
def viz_gpa(gpa,x1range,x2range,dx,alpha,beta=1,thresh=False):
    
    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)
    
    X1grid,X2grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel()]).transpose()
    Xs = np.hstack((Xs, np.ones((Xs.shape[0],1))*alpha))
    
    mus,stds = gpa.predict(Xs, return_std=True)
    mus = mus.flatten()
    
    ys0 = mus
    ysB = mus-beta*stds
    # ys = gpa.predict(Xs)
    ys0 = ys0.reshape(nx2,nx1)
    ysB = ysB.reshape(nx2,nx1)
    
    if thresh:
        ys = (ys0>0).astype(int)+(ysB>0).astype(int)
    else:
        ys = ys0
    
    p = figure(
        x_range=x1range,
        y_range=x2range,
        plot_width=600,
        plot_height=600*nx2//nx1,
        match_aspect=True
    )
    
    color_mapper = LinearColorMapper(palette='Viridis256', low=0, high=2)
    
    if thresh:
        p.image(
            image=[ys],
            x=x1range[0],
            y=x2range[0],
            dw=np.diff(x1range)[0],
            dh=np.diff(x2range)[0],
            color_mapper = color_mapper
        )
    else:
        p.image(
            image=[ys],
            x=x1range[0],
            y=x2range[0],
            dw=np.diff(x1range)[0],
            dh=np.diff(x2range)[0],
            palette='Viridis256'
        )
    return p

In [14]:
p = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,beta=1,thresh=True)
show(p)

---
# Set up sampling method

In [15]:
def gpa_next(gpa,x,y):

    n = gpa.X_train_.shape[1]
    g = skg.GaussianProcessRegressor(
        alpha=1e-6,
        kernel=kernel
    )
    g.fit(
        np.vstack((gpa.X_train_, np.array(x).reshape(-1,n))),
        np.vstack((gpa.y_train_, np.array(y).reshape(-1,1)))
    )
    # print(g.X_train_)
    return g

In [16]:
def p_err(mu,sig):
    return 0.5 - erf(np.abs(mu)/(np.sqrt(2)*sig))

In [17]:
# Given a pair (x,y) of "next" data, return the posterior intagrated Perr
def p_err_next(gpa,x,y):
    
    g = gpa_next(gpa,[x],y)
    
    x1range=[0,2] # x position
    x2range=[-0.5,0.5] # y position
    x3range=[0.1,1] # alpha
    dx = 0.05

    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)
    nx3 = int(np.diff(x3range)[0]/dx)

    X1grid,X2grid,X3grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx),np.arange(*x3range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel(), X2grid.ravel()]).transpose()

    mus, stds = g.predict(Xs, return_std=True)
    mus = mus[:,0]
    p_err_tp1 = np.sum(p_err(mus,stds))
    
    return p_err_tp1

In [18]:
# Given only a "next" x, assume the sampled y will land exactly on the mean and then return the posterior Perr
@jit
def infer_next(x,args):
    
    gpa = args
    
    n = gpa.X_train_.shape[1]
    yi = gpa.predict(x.reshape(-1,n))
    
    p_err_inf = p_err_next(gpa,x,yi)
    
    return p_err_inf

In [19]:
x1range = [0,2]
dx = 0.01

x1 = np.arange(*x1range,dx).reshape(-1,1)
x2 = np.ones_like(x1)*0
x3 = np.ones_like(x1)*0.5

X = np.hstack((x1,x2,x3))
mu, sig = gpa.predict(X, return_std=True)
mu = mu[:,0]

In [20]:
p1 = figure(plot_width=300, plot_height=300)
p1.line(x1.flatten(),mu,line_width=2)

p2 = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5)
p2.line([0,2],[0,0],color="black",line_width=3)

p3 = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,thresh=True)
p3.line([0,2],[0,0],color="black",line_width=3)

grid = gridplot([[p1,p2,p3]])
show(grid)

In [21]:
p_err_inf_vec = np.zeros_like(x1)
for i in range(len(x1)):
    p_err_inf_vec[i] = p_err_next(gpa,np.array([x1[i][0],0,0.5]),mu[i])

mu, sig = gpa.predict(X, return_std=True)
mu = mu.flatten()
    
p1 = figure(plot_width=300, plot_height=200)
p1.line(x1.flatten(),mu,line_width=2)
p1.line(x1.flatten(),mu+sig)
p1.line(x1.flatten(),mu-sig)
p1.circle(
    gpa.X_train_[:,0],
    gpa.y_train_[:,0]
)

p2 = figure(plot_width=300, plot_height=200)
p2.line(x1.flatten(),p_err(mu,sig),line_width=2)
    
p3 = figure(plot_width=300, plot_height=200)
p3.line(x1[:,0],p_err_inf_vec[:,0],line_width=2)

grid = gridplot([[p1],[p2],[p3]])
show(grid)

In [22]:
bounds = [
    [0,2],
    [-0.5,0.5],
    [0.1,1]
]

opt = optimize.differential_evolution(
    infer_next,
    bounds,
    args=[gpa]
)
opt.x

/tmp/ipykernel_6557/3791675807.py:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "infer_next" failed type inference due to: Untyped global name 'p_err_next': Cannot determine Numba type of <class 'function'>

File "../../../../../tmp/ipykernel_6557/3791675807.py", line 10:
<source missing, REPL/exec in use?>

  @jit
/home/kellan/anaconda3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "infer_next" was compiled in object mode without forceobj=True.

File "../../../../../tmp/ipykernel_6557/3791675807.py", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/kellan/anaconda3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs.io/

array([1.6377221 , 0.17985677, 0.17785887])

In [23]:
lam = 1e-2
l = 0.5
kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
gpa_di = skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

# Generate random initial data
n0 = 5
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))
a0 = np.random.uniform(0.1,1,(n0,1))

X = np.hstack((x0,y0,a0))
y = np.array([query_di(x.tolist()) for x in X]).reshape(-1,1)
gpa_di.fit(X,y)

bounds = [
    [0,2],
    [-0.5,0.5],
    [0.1,1]
]

N = 100
for i in tqdm(range(N)):
    
    opt = optimize.differential_evolution(
        infer_next,
        bounds,
        args=[gpa_di]
    )
    
    # opt = optimize.basinhopping(
    #     infer_next,
    #     [1,0,0.5],
        
    x_next = opt.x
    # print(x_next[2])
    y_next = query_di(x_next)
    
    gpa_di = gpa_next(gpa_di,x_next,y_next)
    
    # p = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,thresh=True)
    # show(p)

100%|█████████████████████████████████████████| 100/100 [05:20<00:00,  3.20s/it]


In [24]:
for i in range(5):
    a = (i+1)/5
    p1= viz_gpa(gpa_di,[0,2],[-0.5,0.5],0.01,a,thresh=False)
    p1.circle(gpa_di.X_train_[:,0], gpa_di.X_train_[:,1], color="black")
    p2 = viz_gpa(gpa_di,[0,2],[-0.5,0.5],0.01,a,beta=3,thresh=True)
    show(gridplot([[p1,p2]]))

In [25]:
lam = 1e-2
l = 0.5
kernel = ConstantKernel(1.0, constant_value_bounds="fixed") * RBF(l, length_scale_bounds="fixed")
gpa_uni = skg.GaussianProcessRegressor(
    alpha=lam,
    kernel=kernel
)

# Generate random initial data
n0 = 20
x0 = np.random.uniform(0,2,(n0,1))
y0 = np.random.uniform(-0.5,0.5,(n0,1))
a0 = np.random.uniform(0.1,1,(n0,1))

X = np.hstack((x0,y0,a0))
y = np.array([query_uni(x.tolist()) for x in X]).reshape(-1,1)
gpa_uni.fit(X,y)

bounds = [
    [0,2],
    [-0.5,0.5],
    [0.1,1]
]

N = 50
for i in tqdm(range(N)):
    
    opt = optimize.differential_evolution(
        infer_next,
        bounds,
        args=[gpa_uni]
    )
    
    # opt = optimize.basinhopping(
    #     infer_next,
    #     [1,0,0.5],
        
    x_next = opt.x
    # print(x_next[2])
    y_next = query_uni(x_next)
    
    gpa_uni = gpa_next(gpa_uni,x_next,y_next)
    
    # p = viz_gpa(gpa,[0,2],[-0.5,0.5],0.01,0.5,thresh=True)
    # show(p)

100%|███████████████████████████████████████████| 50/50 [01:54<00:00,  2.29s/it]


In [26]:
for i in range(5):
    a = (i+1)/5
    p1= viz_gpa(gpa_uni,[0,2],[-0.5,0.5],0.01,a,thresh=False)
    p1.circle(gpa_uni.X_train_[:,0], gpa_uni.X_train_[:,1], color="black")
    p2 = viz_gpa(gpa_uni,[0,2],[-0.5,0.5],0.01,a,beta=3,thresh=True)
    show(gridplot([[p1,p2]]))

In [27]:
def save_data(gpa, fname):
    
    x1range=[0,2] # x position
    x2range=[-0.5,0.5] # y position
    x3range=[0.1,1] # alpha
    dx=0.05
    
    nx1 = int(np.diff(x1range)[0]/dx)
    nx2 = int(np.diff(x2range)[0]/dx)

    X1grid,X2grid,X3grid = np.meshgrid(np.arange(*x1range,dx),np.arange(*x2range,dx),np.arange(*x3range,dx))
    Xs = np.vstack([X1grid.ravel(), X2grid.ravel(), X3grid.ravel()]).transpose()

    mus,stds = gpa.predict(Xs, return_std=True)
    mus = mus.flatten()

    data = np.hstack((Xs,mus.reshape(-1,1),stds.reshape(-1,1)))
    df = pd.DataFrame(
        data=data,
        columns=["x1","x2","alpha","mean","stdev"]
    )
    
    df.to_csv(fname)

In [28]:
save_data(gpa_di,"gridplot_di.csv")
save_data(gpa_uni,"gridplot_uni.csv")